<a href="https://colab.research.google.com/github/Boonyaratt/Two-Tower-Recommendation/blob/master/Promotion_LGBMRanker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.Import Library & Datasets

In [5]:
# Import required libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
from datetime import datetime, timedelta
from dateutil import easter
from typing import Dict, Text
import os, kagglehub
import lightgbm as lgb
import heapq
from functools import lru_cache
from collections import defaultdict
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)

print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)

Pandas version: 2.2.2
NumPy version: 2.0.2


In [6]:
!mkdir Dataset

mkdir: cannot create directory ‘Dataset’: File exists


# Datasets

## 2.Display Dataset Information

In [7]:
import os, pathlib
os.environ["KAGGLEHUB_CACHE"] = "/content/Dataset"  # exact case
pathlib.Path("/content/Dataset").mkdir(parents=True, exist_ok=True)

import kagglehub
kaggle_path = kagglehub.dataset_download("frtgnn/dunnhumby-the-complete-journey")
print(kaggle_path)

Using Colab cache for faster access to the 'dunnhumby-the-complete-journey' dataset.
/kaggle/input/dunnhumby-the-complete-journey


In [8]:
# Read all CSV files

path = "/kaggle/input/dunnhumby-the-complete-journey/"
# path = "/content/Dataset/datasets/frtgnn/dunnhumby-the-complete-journey/versions/1/"

campaign_desc = pd.read_csv(path + "campaign_desc.csv")
campaign_table = pd.read_csv(path + "campaign_table.csv")
coupon_redempt = pd.read_csv(path + "coupon_redempt.csv") ## 1 counpon_unc can have multiple product_id: 556 nunique from 2318
coupon = pd.read_csv(path + "coupon.csv")
product = pd.read_csv(path + "product.csv")
transaction_data = pd.read_csv(path + "transaction_data.csv")
# Check the dataframes
print("DataFrames loaded successfully!")
print(f"campaign_desc shape: {campaign_desc.shape}")
print(f"campaign_table shape: {campaign_table.shape}")
print(f"coupon_redempt shape: {coupon_redempt.shape}")
print(f"coupon shape: {coupon.shape}")
print(f"product shape: {product.shape}")
print(f"transaction_data shape: {transaction_data.shape}")

# Show headers for each dataset
print("Headers for each dataset:")
print("=" * 50)

datasets = {
    'campaign_desc': campaign_desc,
    'campaign_table': campaign_table,
    'coupon_redempt': coupon_redempt,
    'coupon': coupon,
    'product': product,
    'transaction_data': transaction_data
}

for name, df in datasets.items():
    print(f"\n{name}:")
    print(f"Columns: {list(df.columns)}")



DataFrames loaded successfully!
campaign_desc shape: (30, 4)
campaign_table shape: (7208, 3)
coupon_redempt shape: (2318, 4)
coupon shape: (124548, 3)
product shape: (92353, 7)
transaction_data shape: (2595732, 12)
Headers for each dataset:

campaign_desc:
Columns: ['DESCRIPTION', 'CAMPAIGN', 'START_DAY', 'END_DAY']

campaign_table:
Columns: ['DESCRIPTION', 'household_key', 'CAMPAIGN']

coupon_redempt:
Columns: ['household_key', 'DAY', 'COUPON_UPC', 'CAMPAIGN']

coupon:
Columns: ['COUPON_UPC', 'PRODUCT_ID', 'CAMPAIGN']

product:
Columns: ['PRODUCT_ID', 'MANUFACTURER', 'DEPARTMENT', 'BRAND', 'COMMODITY_DESC', 'SUB_COMMODITY_DESC', 'CURR_SIZE_OF_PRODUCT']

transaction_data:
Columns: ['household_key', 'BASKET_ID', 'DAY', 'PRODUCT_ID', 'QUANTITY', 'SALES_VALUE', 'STORE_ID', 'RETAIL_DISC', 'TRANS_TIME', 'WEEK_NO', 'COUPON_DISC', 'COUPON_MATCH_DISC']


In [9]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   PRODUCT_ID            92353 non-null  int64 
 1   MANUFACTURER          92353 non-null  int64 
 2   DEPARTMENT            92353 non-null  object
 3   BRAND                 92353 non-null  object
 4   COMMODITY_DESC        92353 non-null  object
 5   SUB_COMMODITY_DESC    92353 non-null  object
 6   CURR_SIZE_OF_PRODUCT  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


###2-1. Sample Selection

####Houeholds Key Selection

In [10]:
# keep top-N households by txn count (สำหรับเทสต์)
N_HH = 500
top_hh = (transaction_data["household_key"].value_counts().head(N_HH).index.astype(str))
transaction_data = transaction_data[transaction_data["household_key"].astype(str).isin(top_hh)].copy()
campaign_table   = campaign_table[campaign_table["household_key"].astype(str).isin(set(top_hh))].copy()
coupon_redempt   = coupon_redempt[coupon_redempt["household_key"].astype(str).isin(set(top_hh))].copy()

#### Window day Selection

In [11]:
# === OPTIONAL: Dataset windowing to speed up ===
# ใช้ช่วงวันล่าสุด N วัน (เช่น 180 วันล่าสุด) หรือกำหนดช่วงเองก็ได้
WINDOW_DAYS = 180

min_day_all = int(transaction_data["DAY"].min())
max_day_all = int(transaction_data["DAY"].max())
start_day = max(min_day_all, max_day_all - WINDOW_DAYS + 1)
end_day   = max_day_all

# 1) ตัดธุรกรรมให้เหลือเฉพาะช่วงวัน
transaction_data = transaction_data[(transaction_data["DAY"] >= start_day) & (transaction_data["DAY"] <= end_day)].copy()

# 2) จำกัด household ให้เหลือเฉพาะที่ยังมีธุรกรรมในช่วงวัน
keep_hh = set(transaction_data["household_key"].astype(str).unique())
campaign_table = campaign_table[campaign_table["household_key"].astype(str).isin(keep_hh)].copy()
coupon_redempt = coupon_redempt[coupon_redempt["household_key"].astype(str).isin(keep_hh)].copy()

# 3) จำกัดแคมเปญให้ทับซ้อนช่วงวัน (START_DAY..END_DAY) กับหน้าต่างที่เลือก
campaign_desc = campaign_desc[
    (campaign_desc["END_DAY"] >= start_day) & (campaign_desc["START_DAY"] <= end_day)
].copy()
keep_campaigns = set(campaign_desc["CAMPAIGN"].astype(str).unique())
campaign_table = campaign_table[campaign_table["CAMPAIGN"].astype(str).isin(keep_campaigns)].copy()
coupon_redempt = coupon_redempt[coupon_redempt["CAMPAIGN"].astype(str).isin(keep_campaigns)].copy()
coupon = coupon[coupon["CAMPAIGN"].astype(str).isin(keep_campaigns)].copy()

# 4) (ออปชัน) จำกัดสินค้าให้เหลือเฉพาะที่ปรากฏในธุรกรรมช่วงนี้ เพื่อยุบมิติ OHE
keep_products = set(transaction_data["PRODUCT_ID"].astype(str).unique())
product = product[product["PRODUCT_ID"].astype(str).isin(keep_products)].copy()

print(f"Using DAY window [{start_day}, {end_day}]")
print("transaction_data:", transaction_data.shape, "campaign_desc:", campaign_desc.shape,
      "campaign_table:", campaign_table.shape, "coupon_redempt:", coupon_redempt.shape,
      "coupon:", coupon.shape, "product:", product.shape)

Using DAY window [532, 711]
transaction_data: (367796, 12) campaign_desc: (13, 4) campaign_table: (2168, 3) coupon_redempt: (1177, 4) coupon: (87484, 3) product: (36206, 7)


In [12]:
transaction_data['household_key'] = transaction_data['household_key'].astype(str)
transaction_data['STORE_ID'] = transaction_data['STORE_ID'].astype(str)
transaction_data['PRODUCT_ID'] = transaction_data['PRODUCT_ID'].astype(str)
transaction_data['BASKET_ID'] = transaction_data['BASKET_ID'].astype(str)

# FIXED: Use .astype(str) instead of str() for the entire column
coupon['COUPON_UPC'] = coupon['COUPON_UPC'].astype(str)  # Changed this line
coupon['PRODUCT_ID'] = coupon['PRODUCT_ID'].astype(str)
coupon['CAMPAIGN'] = coupon['CAMPAIGN'].astype(str)

campaign_desc['CAMPAIGN'] = campaign_desc['CAMPAIGN'].astype(str)

coupon_redempt['household_key'] = coupon_redempt['household_key'].astype(str)
coupon_redempt['COUPON_UPC'] = coupon_redempt['COUPON_UPC'].astype(str)
coupon_redempt['CAMPAIGN'] = coupon_redempt['CAMPAIGN'].astype(str)

campaign_table['CAMPAIGN'] = campaign_table['CAMPAIGN'].astype(str)
campaign_table['household_key'] = campaign_table['household_key'].astype(str)

product['PRODUCT_ID'] = product['PRODUCT_ID'].astype(str)
product['DEPARTMENT'] = product['DEPARTMENT'].astype(str)
product['MANUFACTURER'] = product['MANUFACTURER'].astype(str)

##3.Data Preparation

###3-1. Merging

In [13]:
# Define promotion key
coupon['COUPON_UPC'] = coupon['COUPON_UPC'].astype(str)
coupon['CAMPAIGN'] = coupon['CAMPAIGN'].astype(str)
campaign_desc['CAMPAIGN'] = campaign_desc['CAMPAIGN'].astype(str)
coupon_redempt['COUPON_UPC'] = coupon_redempt['COUPON_UPC'].astype(str)
coupon_redempt['CAMPAIGN'] = coupon_redempt['CAMPAIGN'].astype(str)

coupon['PROMO_KEY'] = coupon['COUPON_UPC'] + '|' + coupon['CAMPAIGN']
coupon_redempt['PROMO_KEY'] = coupon_redempt['COUPON_UPC'] + '|' + coupon_redempt['CAMPAIGN']

promo_catalog = coupon.merge(
    campaign_desc[['CAMPAIGN','START_DAY','END_DAY']],
    on='CAMPAIGN', how='left'
)[['PROMO_KEY','COUPON_UPC','CAMPAIGN','PRODUCT_ID','START_DAY','END_DAY']]

# All campaigns per household
hh_campaigns = campaign_table[['household_key','CAMPAIGN']].drop_duplicates()
hh_promos = hh_campaigns.merge(
    promo_catalog[['CAMPAIGN','PROMO_KEY','START_DAY','END_DAY']],
    on='CAMPAIGN', how='left'
).dropna(subset=['PROMO_KEY'])

# Promo -> set of products
promo_to_products = (
    coupon[['COUPON_UPC','CAMPAIGN','PRODUCT_ID']]
    .assign(PROMO_KEY=lambda df: df['COUPON_UPC'] + '|' + df['CAMPAIGN'])
    .groupby('PROMO_KEY')['PRODUCT_ID']
    .apply(lambda s: set(s.astype(str)))
    .to_dict()
)

# One-hot for promo metadata aggregated from product
coupon_prod = coupon.merge(
    product[['PRODUCT_ID','DEPARTMENT','BRAND']],
    on='PRODUCT_ID', how='left'
).assign(PROMO_KEY=lambda df: df['COUPON_UPC'] + '|' + df['CAMPAIGN'])
coupon_prod['DEPARTMENT'] = coupon_prod['DEPARTMENT'].fillna('UNK')
coupon_prod['BRAND'] = coupon_prod['BRAND'].fillna('UNK')
promo_ohe = pd.get_dummies(
    coupon_prod[['PROMO_KEY','DEPARTMENT','BRAND']], columns=['DEPARTMENT','BRAND'], drop_first=False
).groupby('PROMO_KEY').max().astype(bool)

# Promo popularity by redemptions
promo_pop = coupon_redempt['PROMO_KEY'].value_counts().to_dict()

# Redemptions per (household, DAY) as positives lookup
redempt_by_hh_day = (
    coupon_redempt.groupby(['household_key','DAY'])['PROMO_KEY']
    .apply(set).to_dict()
)

In [14]:
coupon_redempt= coupon_redempt.merge(
    coupon[['COUPON_UPC', 'PRODUCT_ID']],
    on='COUPON_UPC',
    how='left'
)

coupon= coupon.merge(
    campaign_desc[['START_DAY', 'END_DAY', 'CAMPAIGN']],
    on='CAMPAIGN',
    how='left'
)

coupon = coupon.merge(
    product[['PRODUCT_ID', 'DEPARTMENT', 'MANUFACTURER']],
    on='PRODUCT_ID',
    how='left'
)


###3-2. Create Time *Features*

In [15]:
# Create timestamp feature from DAY and TRANS_TIME
def create_timestamp(day, trans_time):
    base_date = datetime(2010, 3, 24)  # Arbitrary base date
    date_part = base_date + timedelta(days=int(day) - 1)

    # Convert TRANS_TIME to time
    hours = int(trans_time) // 100
    minutes = int(trans_time) % 100
    time_part = timedelta(hours=hours, minutes=minutes)

    return date_part + time_part

# Add timestamp to transaction_data
transaction_data['timestamp'] = transaction_data.apply(
    lambda row: create_timestamp(row['DAY'], row['TRANS_TIME']), axis=1
)

ts = transaction_data["timestamp"]
transaction_data["hour"]        = ts.dt.hour.astype("Int64")

transaction_data["dayofweek"]   = ts.dt.dayofweek.astype("Int64")    # 0=Mon .. 6=Sun
transaction_data["is_weekend"]  = transaction_data["dayofweek"].isin([5,6]).astype("int64")

transaction_data["day"]         = ts.dt.day.astype("Int64")
transaction_data["week"]        = ts.dt.isocalendar().week.astype("Int64")
transaction_data["month"]       = ts.dt.month.astype("Int64")
transaction_data["quarter"]     = ts.dt.quarter.astype("Int64")
transaction_data["year"]        = ts.dt.year.astype("Int64")

def make_part_of_day(hhmm_int):
    hh = hhmm_int // 100
    return (
        "night" if hh < 6 else
        "morning" if hh < 12 else
        "afternoon" if hh < 18 else
        "evening"
    )

transaction_data["part_of_day"] = transaction_data["TRANS_TIME"].astype(int).apply(make_part_of_day)
transaction_data = transaction_data.merge(
        product[['PRODUCT_ID', 'DEPARTMENT',"MANUFACTURER"]],
        on='PRODUCT_ID',
        how='left'
    )

transaction_data = transaction_data.merge(
    coupon_redempt,
    on=['household_key', 'DAY', 'PRODUCT_ID'],
    how='left'
)


In [16]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368093 entries, 0 to 368092
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   household_key      368093 non-null  object        
 1   BASKET_ID          368093 non-null  object        
 2   DAY                368093 non-null  int64         
 3   PRODUCT_ID         368093 non-null  object        
 4   QUANTITY           368093 non-null  int64         
 5   SALES_VALUE        368093 non-null  float64       
 6   STORE_ID           368093 non-null  object        
 7   RETAIL_DISC        368093 non-null  float64       
 8   TRANS_TIME         368093 non-null  int64         
 9   WEEK_NO            368093 non-null  int64         
 10  COUPON_DISC        368093 non-null  float64       
 11  COUPON_MATCH_DISC  368093 non-null  float64       
 12  timestamp          368093 non-null  datetime64[ns]
 13  hour               368093 non-null  Int64   

### 3-3. Global Constants

In [17]:
RANDOM_STATE = 42
TOPK_CANDS = 20         # จำนวน candidates ต่อ query ที่จะป้อนเข้า ranker
EVAL_AT = [5, 10]          # NDCG@5, NDCG@10

###3-4. Create *Sequential Orders* per household

Aggregate transactions into baskets per `(household_key, BASKET_ID)`:
- `items`: set of `PRODUCT_ID`s in basket.
- `basket_day`: the day of the basket.
- `next_items` / `next_day`: next basket’s items/day (for sequence context).

#### Why
Provides user history over time for candidate generation without leakage.

In [18]:
transaction_data = transaction_data.sort_values(["household_key", "timestamp", "BASKET_ID"])
orders = (
    transaction_data.groupby(["household_key", "BASKET_ID"])
    .agg(items=("PRODUCT_ID", lambda s: set(s.tolist())),
         basket_day=("DAY", "min"))
    .reset_index()
)

# for now 'next_items' and 'next'day' is not neccesary for training due to we use real-time suggestion not next-day suggestion
orders["next_items"] = orders.groupby("household_key")["items"].shift(-1)
orders["next_day"] = orders.groupby("household_key")["basket_day"].shift(-1)
orders = orders.dropna(subset=["next_day"]).reset_index(drop=True)

In [19]:
orders

,household_key,BASKET_ID,items,basket_day,next_items,next_day
0,1,35840912229,"{1088462, 9527558, 913270, 7130924, 995242, 11...",535,{819312},536.0
1,1,35841522378,{819312},536,{9527290},537.0
2,1,35865806127,{9527290},537,{1069297},538.0
3,1,35877676640,{1069297},538,"{844818, 883039, 6034857, 12385374, 9527290, 1...",540.0
4,1,36027807102,"{844818, 883039, 6034857, 12385374, 9527290, 1...",540,"{961554, 856942, 1069575, 5577022}",546.0
...,...,...,...,...,...,...
32501,998,41918011754,"{15830211, 1091365, 1134152, 1007136}",683,"{899624, 849843, 995242, 845208, 981760}",691.0
32502,998,42031013564,"{899624, 849843, 995242, 845208, 981760}",691,{962568},693.0
32503,998,42049756099,{962568},693,"{865874, 843338, 12524086}",694.0
32504,998,42063285897,"{865874, 843338, 12524086}",694,"{6533765, 1018085, 1049922}",703.0


###3-5.Candidate Generation Baseline

#### Purpose
A baseline function to score active promos by overlap with user history plus small popularity prior.

#### Note
Kept for reference; the pipeline uses the optimized inverted-index approach in the next cell.

In [20]:
# check if that id got the promotion on specific day
def get_active_promos_for_household_on_day(hh: str, day: int) -> list[str]:
    rows = hh_promos[hh_promos['household_key'] == hh]
    rows = rows[(rows['START_DAY'] <= int(day)) & (int(day) <= rows['END_DAY'])]
    return rows['PROMO_KEY'].tolist()

# create candidate list for hh-id in specific day
def build_promo_candidates(household_key: str, user_hist_items: set[str], day: int, topk: int = TOPK_CANDS) -> list[str]:
    active = get_active_promos_for_household_on_day(household_key, day) #check if it active for hh_id in that day
    if not active:
        return []
    # score by overlap with user history + 0.001 x global popularity
    def score(pkey: str) -> float:
        prods = promo_to_products.get(pkey, set())
        overlap = len(prods & set(user_hist_items or []))
        pop = promo_pop.get(pkey, 0)
        return overlap + 0.001 * pop
    ranked = sorted(active, key=score, reverse=True)
    return ranked[:topk]

###3-6. Build (query, candidate) labeled rows

#### Purpose
Efficiently create `pair_df` of (query, candidate promo, label) using:
- One query per household per day (`orders_day`).
- `hist_exclusive` (history before the day) to avoid leakage.
- Active promo filtering with LRU cache.
- Inverted index from `PRODUCT_ID -> {PROMO_KEY}` to propose promos overlapping user history.
- `heapq.nlargest` for Top-K by overlap + tiny popularity prior.
- Fallback: fill remaining slots with most popular active promos.
- Labels: same-day redemptions are forced into candidates and marked `label=1`.

#### Outputs
- `pair_df`: columns `household_key`, `BASKET_ID`, `PROMO_KEY`, `label`.

In [21]:
# Expect runtime : ~2 mi
# 1 query per 1 household_id per day
orders_day = (
    orders.sort_values(["household_key","basket_day","BASKET_ID"])
    .groupby(["household_key","basket_day"], as_index=False)
    .first()  # use only first basket_id on that day
    .rename(columns={"basket_day":"day_t"})
)
# สร้าง hist_exclusive (ประวัติก่อนตะกร้าปัจจุบัน)
def _add_hist_exclusive_per_day(grp):
    hist = set()
    res = []
    for s in grp["items"]:
        res.append(hist.copy())
        hist |= s
    out = grp.copy()
    out["hist_exclusive"] = res
    return out

orders_fast = (
    orders_day.sort_values(["household_key","day_t"])
    .groupby("household_key", group_keys=False)
    .apply(_add_hist_exclusive_per_day)
    .reset_index(drop=True)
)

# 1) Pre-index promos per household (active filter เร็ว)
hh_to_promos = {
    hh: grp[["START_DAY","END_DAY","PROMO_KEY"]].to_numpy()
    for hh, grp in hh_promos.groupby("household_key", sort=False)
}

@lru_cache(maxsize=1_000_000)
def get_active_promos_tuple(hh: str, day: int) -> tuple:
    arr = hh_to_promos.get(hh)
    if arr is None:
        return ()
    d = int(day)
    mask = (arr[:,0] <= d) & (d <= arr[:,1])
    if not mask.any():
        return ()
    return tuple(arr[mask][:,2])

# 2) Inverted index: PRODUCT_ID -> set(PROMO_KEY)
product_to_promos = defaultdict(set)
for pkey, prods in promo_to_products.items():
    for pid in prods:
        product_to_promos[pid].add(pkey)

# 3) แคช active ที่เรียงตามความนิยมไว้เติม fallback
@lru_cache(maxsize=1_000_000)
def get_active_promos_sorted_by_pop(hh: str, day: int) -> tuple:
    active = get_active_promos_tuple(hh, day)
    if not active:
        return ()
    # เรียงตาม popularity จากมากไปน้อย
    return tuple(sorted(active, key=lambda k: promo_pop.get(k, 0), reverse=True))

# Locals
empty_set = frozenset()
promo_pop_local = promo_pop
promo_to_products_local = promo_to_products
redempt_by_hh_day_local = redempt_by_hh_day
TOPK = TOPK_CANDS  # ปรับลงเป็น 20 ชั่วคราวได้ตอนเทสต์

# 4) Build columns
col_hh, col_bid, col_promo, col_label = [], [], [], []

for r in orders_fast.itertuples(index=False):
    hh = r.household_key
    bid = r.BASKET_ID
    day_t = int(r.day_t)
    user_hist = r.hist_exclusive if isinstance(r.hist_exclusive, set) else set(r.hist_exclusive)

    active = get_active_promos_tuple(hh, day_t)
    if not active:
        continue
    active_set = set(active)

    # 4.1) ใช้ inverted index สร้างชุด candidate จากสินค้าที่ user เคยซื้อ (ลดขนาดอย่างมาก)
    cand_from_hist = set()
    # จำกัดขนาดประวัติ (เช่น 200 ชิ้นล่าสุด) เพื่อลดเวลา
    if len(user_hist) > 200:
        # แปลงเป็น list แล้วหยิบต้นๆ พอเป็นตัวแทน (hist_exclusive เป็น set จึงไม่มีลำดับที่แท้)
        user_hist_iter = list(user_hist)[:200]
    else:
        user_hist_iter = user_hist

    for pid in user_hist_iter:
        cand_from_hist |= product_to_promos.get(pid, empty_set)

    # เฉพาะโปรโมชันที่ active จริงในวันนั้น
    cand_from_hist &= active_set

    # 4.2) ให้คะแนนเฉพาะชุดที่ overlap (เล็กลงมาก) แล้วคัด TopK
    def score(pkey: str) -> float:
        prods = promo_to_products_local.get(pkey, empty_set)
        return (len(prods & user_hist) if prods else 0.0) + 0.001 * promo_pop_local.get(pkey, 0)

    if cand_from_hist:
        ranked_overlap = heapq.nlargest(TOPK, cand_from_hist, key=score)
    else:
        ranked_overlap = []

    # 4.3) เติม fallback ด้วย active ที่นิยมสูง (ไม่ต้องคำนวณ score เพิ่ม)
    if len(ranked_overlap) < TOPK:
        need = TOPK - len(ranked_overlap)
        fallback = []
        seen = set(ranked_overlap)
        for p in get_active_promos_sorted_by_pop(hh, day_t):
            if p not in seen:
                seen.add(p)
                fallback.append(p)
                if len(fallback) >= need:
                    break
        cands_base = ranked_overlap + fallback
    else:
        cands_base = ranked_overlap

    # 4.4) ใส่ positives (same-day) ให้อยู่ต้นๆ แล้ว dedupe
    positives = redempt_by_hh_day_local.get((hh, day_t), empty_set)
    if positives:
        pos_list = list(positives)
        if pos_list:
            seen = set()
            cands = []
            for p in pos_list:
                if p not in seen:
                    seen.add(p); cands.append(p)
            for p in cands_base:
                if p not in seen:
                    seen.add(p); cands.append(p)
            cands = cands[:TOPK]
        else:
            cands = cands_base[:TOPK]
    else:
        cands = cands_base[:TOPK]

    if not cands:
        continue

    is_pos = positives.__contains__
    col_hh.extend([hh] * len(cands))
    col_bid.extend([int(bid)] * len(cands))
    col_promo.extend(cands)
    col_label.extend([1 if is_pos(p) else 0 for p in cands])

pair_df = pd.DataFrame({
    "household_key": col_hh,
    "BASKET_ID": col_bid,
    "PROMO_KEY": col_promo,
    "label": col_label
})
pair_df.head(3)

,household_key,BASKET_ID,PROMO_KEY,label
0,1,35840912229,10000085429|13,0
1,1,35840912229,10000085425|13,0
2,1,35840912229,51600070033|13,0


In [34]:
orders_fast

,household_key,day_t,BASKET_ID,items,next_items,next_day,hist_exclusive
0,1,535,35840912229,"{1088462, 9527558, 913270, 7130924, 995242, 11...",{819312},536.0,{}
1,1,536,35841522378,{819312},{9527290},537.0,"{1088462, 9527558, 913270, 7130924, 995242, 11..."
2,1,537,35865806127,{9527290},{1069297},538.0,"{819312, 1088462, 9527558, 913270, 7130924, 99..."
3,1,538,35877676640,{1069297},"{844818, 883039, 6034857, 12385374, 9527290, 1...",540.0,"{819312, 1088462, 9527558, 913270, 7130924, 99..."
4,1,540,36027807102,"{844818, 883039, 6034857, 12385374, 9527290, 1...","{961554, 856942, 1069575, 5577022}",546.0,"{819312, 1088462, 9527558, 1069297, 913270, 71..."
...,...,...,...,...,...,...,...
25288,998,683,41918011754,"{15830211, 1091365, 1134152, 1007136}","{899624, 849843, 995242, 845208, 981760}",691.0,"{936594, 948119, 9834000, 16219317, 1071330, 1..."
25289,998,691,42031013564,"{899624, 849843, 995242, 845208, 981760}",{962568},693.0,"{936594, 948119, 9834000, 16219317, 1071330, 1..."
25290,998,693,42049756099,{962568},"{865874, 843338, 12524086}",694.0,"{936594, 948119, 9834000, 16219317, 1071330, 1..."
25291,998,694,42063285897,"{865874, 843338, 12524086}","{6533765, 1018085, 1049922}",703.0,"{936594, 948119, 9834000, 16219317, 1071330, 1..."


In [36]:
orders_fast['hist_exclusive'].iloc[1]

{'1088462',
 '1124029',
 '5577022',
 '7130924',
 '888104',
 '913270',
 '9527558',
 '995242'}

#### Downsample negatives (class balancing for training)

#### Purpose
Reduce class imbalance by capping negatives per query group and optionally removing groups with no positives.

#### Why
Improves training efficiency and precision without altering validation distribution.

#### Outputs
- Updated `pair_df` with fewer negatives; positives preserved.

In [38]:
def downsample_negatives(df, max_neg_per_pos=30, drop_neg_only_groups=True):
    def _sub(g):
        pos = g[g.label == 1]
        neg = g[g.label == 0]
        if len(pos) == 0:
            # ตัดกลุ่มนี้ทิ้งจาก train
            return g.iloc[0:0] if drop_neg_only_groups else neg.sample(min(50, len(neg)), random_state=42)
        keep_neg = neg.sample(min(len(pos) * max_neg_per_pos, len(neg)), random_state=42)
        return pd.concat([pos, keep_neg], axis=0)
    return df.groupby(['household_key','BASKET_ID'], group_keys=False).apply(_sub).reset_index(drop=True)

pair_df = downsample_negatives(pair_df, max_neg_per_pos=30)

In [39]:
pair_df['label'].value_counts()

,count
label,
0,7285
1,821


In [40]:
pair_df[pair_df["label"] == 1 ]

,household_key,BASKET_ID,PROMO_KEY,label
0,1,40666800522,10000085476|18,1
1,1,40666800522,54200029176|18,1
20,104,40533315328,53450058040|17,1
40,104,40703416038,54280011660|17,1
41,104,40703416038,53800030050|17,1
...,...,...,...,...
8049,982,41338129604,51800015050|18,1
8050,982,41338129604,51600070033|18,1
8066,99,36029865630,10000089077|13,1
8086,99,41258861040,54060060076|18,1


### 3-7.PRomotion Feature Engineering

#### Purpose
Construct features per (query, promo):
- `feat_promo_pop`: promo redemption popularity.
- `feat_user_affinity`: sum of user’s purchases over products covered by the promo.
- `promo_ohe` one-hot: aggregated `DEPARTMENT_*` and `BRAND_*`.

Create `X` (features), `y` (labels), and `qid` (group id per query).

#### Outputs
- Feature matrix `X`, labels `y`, and grouping keys `qid`.

In [41]:
# === REPLACE Cell 6: promotion feature engineering ===
# Numeric: promo popularity
feat = pair_df.copy()
feat["feat_promo_pop"] = feat["PROMO_KEY"].map(promo_pop).fillna(0).astype(float)

# User x promo affinity: sum of user purchases over coupon-covered products
# Build mapping DataFrames for efficient aggregation
promo_prod_map = (
    coupon.assign(PROMO_KEY=lambda df: df['COUPON_UPC'] + '|' + df['CAMPAIGN'])
    [["PROMO_KEY","PRODUCT_ID"]].drop_duplicates()
)

user_product_ct = (
    transaction_data.groupby(["household_key","PRODUCT_ID"]).size()
    .reset_index(name="user_prod_ct")
)

aff = (
    feat[["household_key","PROMO_KEY"]].drop_duplicates()
    .merge(promo_prod_map, on="PROMO_KEY", how="left")
    .merge(user_product_ct, on=["household_key","PRODUCT_ID"], how="left")
    .fillna({"user_prod_ct":0})
    .groupby(["household_key","PROMO_KEY"])["user_prod_ct"].sum()
    .reset_index()
    .rename(columns={"user_prod_ct":"feat_user_affinity"})
)

feat = feat.merge(aff, on=["household_key","PROMO_KEY"], how="left")
feat["feat_user_affinity"] = feat["feat_user_affinity"].fillna(0).astype(float)

# One-hot: aggregated DEPARTMENT/BRAND at promo level (precomputed in promo_ohe)
X_cats = promo_ohe.reindex(feat["PROMO_KEY"]).fillna(False).reset_index(drop=True)

# Assemble X, y
X_num = feat[["feat_promo_pop","feat_user_affinity"]].reset_index(drop=True)
X = pd.concat([X_num, X_cats], axis=1)
y = feat["label"].values

# qid for LGBMRanker
feat["qid"] = feat["household_key"].astype(str) + "_" + feat["BASKET_ID"].astype(str)

In [42]:
# === Cell 7: group-aware split ===
qids = feat["qid"].unique()
q_train, q_valid = train_test_split(qids, test_size=0.2, random_state=RANDOM_STATE)

train_mask = feat["qid"].isin(q_train)
valid_mask = feat["qid"].isin(q_valid)

X_tr, y_tr = X[train_mask], y[train_mask]
X_va, y_va = X[valid_mask], y[valid_mask]

grp_tr = feat.loc[train_mask].groupby("qid").size().tolist()
grp_va = feat.loc[valid_mask].groupby("qid").size().tolist()


In [43]:
X

,feat_promo_pop,feat_user_affinity,DEPARTMENT_CHEF SHOPPE,DEPARTMENT_COSMETICS,DEPARTMENT_COUP/STR & MFG,DEPARTMENT_DELI,DEPARTMENT_DRUG GM,DEPARTMENT_FLORAL,DEPARTMENT_FROZEN GROCERY,DEPARTMENT_GARDEN CENTER,...,DEPARTMENT_PRODUCE,DEPARTMENT_RX,DEPARTMENT_SALAD BAR,DEPARTMENT_SEAFOOD,DEPARTMENT_SEAFOOD-PCKGD,DEPARTMENT_TRAVEL & LEISUR,DEPARTMENT_UNK,BRAND_National,BRAND_Private,BRAND_UNK
0,5.0,58.0,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,True,True,True,True
1,7.0,4.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
2,46.0,63.0,True,False,False,True,True,True,True,True,...,True,False,True,False,True,True,True,True,True,True
3,18.0,30.0,False,False,False,True,False,False,False,False,...,False,False,True,True,True,False,True,True,True,True
4,1.0,1.0,False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101,9.0,0.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
8102,19.0,0.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
8103,12.0,0.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
8104,9.0,0.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True


##4.Train LGBMRanker (LambdaRank)

#### Purpose
Train a gradient-boosted ranking model with LambdaRank objective and NDCG metric.

#### Notes
- Uses early stopping callbacks.
- Consider `n_jobs=-1` to use all CPU cores.
- Training groups provided via `grp_tr`/`grp_va`.

In [44]:
# === Cell 8: train LGBMRanker (fixed: use callbacks instead of verbose) ===
import lightgbm as lgb

ranker = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    num_leaves=63,
    learning_rate=0.05,
    n_estimators=500,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    # ถ้าต้องการลด warning จาก lightgbm เอง อาจใช้ verbosity=-1 (คนละเรื่องกับ progress log)
    # verbosity=-1,
)

# ใช้ callbacks แทน verbose
callbacks = [
    lgb.log_evaluation(period=50),                    # print metric ทุก ๆ 50 รอบ
    lgb.early_stopping(stopping_rounds=50,           # early stop ถ้าไม่ดีขึ้น 50 รอบ
                       first_metric_only=True)        # โฟกัสเมตริกตัวแรก (ndcg)
]

ranker.fit(
    X_tr, y_tr,
    group=grp_tr,
    eval_set=[(X_va, y_va)],
    eval_group=[grp_va],
    eval_at=EVAL_AT,          # เช่น [5,10] → NDCG@5, NDCG@10
    callbacks=callbacks       # << เปลี่ยนจาก verbose=50 มาเป็น callbacks
)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001706 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 238
[LightGBM] [Info] Number of data points in the train set: 6479, number of used features: 29
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

LGBMRanker(colsample_bytree=0.8, learning_rate=0.05, metric='ndcg',
           n_estimators=500, num_leaves=63, objective='lambdarank',
           random_state=42, subsample=0.8)

##5.Evaluation
#### Purpose
Compute Recall@K and Precision@K across queries by taking the top-K per `qid`.

#### Outputs
- Printed Recall@10/3/1 and Precision@10/3/1 for validation.

In [45]:
# === REPLACE Cell 9: inference & within-group ranking (promotions) ===
valid_frame = feat[valid_mask].copy()
valid_frame["score"] = ranker.predict(X_va)

valid_frame["rank"] = valid_frame.groupby("qid")["score"].rank(ascending=False, method="first")
topk = (
    valid_frame.sort_values(["qid","rank"])
    .groupby("qid")
    .head(10)[["qid","PROMO_KEY","score","rank","label"]]
)

topk.head(10)

,qid,PROMO_KEY,score,rank,label
183,113_40234737911,53700048076|16,0.111471,1.0,0
180,113_40234737911,53500000076|16,0.094767,2.0,1
189,113_40234737911,10000085482|16,0.042429,3.0,0
187,113_40234737911,52270011076|16,-0.089273,4.0,0
190,113_40234737911,50099700082|16,-0.089273,5.0,0
185,113_40234737911,57124921076|16,-0.121363,6.0,0
186,113_40234737911,53600030082|16,-0.121363,7.0,0
191,113_40234737911,57418211676|16,-0.121363,8.0,0
184,113_40234737911,10000085436|16,-0.132963,9.0,0
192,113_40234737911,55310000076|16,-0.132963,10.0,0


In [46]:
# === Cell 10: simple Recall@K / Precision@K ===
def recall_precision_at_k(df, k=10):
    got = (df.sort_values(["qid","score"], ascending=[True, False])
             .groupby("qid").head(k))
    # recall@k = #positive ที่ดึงติด / #positive ทั้งหมดในกลุ่ม
    pos_per_q = df.groupby("qid")["label"].sum()
    hit_per_q = got.groupby("qid")["label"].sum()
    recall = (hit_per_q / pos_per_q.replace(0, np.nan)).mean()  # เฉลี่ยเฉพาะกลุ่มที่มี positive
    precision = got.groupby("qid")["label"].mean().mean()       # เฉลี่ย precision ต่อ qid
    return recall, precision


In [47]:
recall10, precision10 = recall_precision_at_k(valid_frame, k=10)
print(f"Recall@10={recall10:.4f}  Precision@10={precision10:.4f}")

recall3, precision3 = recall_precision_at_k(valid_frame, k=3)
print(f"Recall@3={recall3:.4f}  Precision@3={precision3:.4f}")

recall1, precision1 = recall_precision_at_k(valid_frame, k=1)
print(f"Recall@1={recall1:.4f}  Precision@1={precision1:.4f}")

Recall@10=0.8316  Precision@10=0.1518
Recall@3=0.4566  Precision@3=0.2530
Recall@1=0.1950  Precision@1=0.3133


In [50]:
import os, json, joblib
import lightgbm as lgb

# เซฟ (ตามที่คุณทำไว้)
os.makedirs("artifacts", exist_ok=True)
joblib.dump(ranker, "artifacts/lgbm_ranker.pkl")
ranker.booster_.save_model("artifacts/lgbm_ranker.txt")
with open("artifacts/feature_columns.json", "w", encoding="utf-8") as f:
    json.dump(list(X.columns), f, ensure_ascii=False, indent=2)
with open("artifacts/promo_ohe_columns.json", "w", encoding="utf-8") as f:
    json.dump(list(promo_ohe.columns), f, ensure_ascii=False, indent=2)

# โหลดกลับมาทดสอบบนชุด validation
ranker = joblib.load("artifacts/lgbm_ranker.pkl")
with open("artifacts/feature_columns.json", "r", encoding="utf-8") as f:
    feature_cols = json.load(f)

# ใช้ X_va เป็น X_new และจัดคอลัมน์ให้ตรง
X_new = X_va.reindex(columns=feature_cols, fill_value=False)
scores = ranker.predict(X_new)
scores[:10]

array([-0.20603268,  0.09109491,  0.05209724, -0.00341201,  0.14602422,
       -0.06572048, -0.31430737, -0.22797496, -0.23091383, -0.18384093])